In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('merge_data_update.csv', index_col=0)

In [3]:
tdata = data.copy()

In [4]:
tdata

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,ClassPr_S,ClassSc_S,SchD_S,TxD_S,LndAc_S,ImpAc_S,OthAc_S,HSTtl_D,HSTtl_S,GLA
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,...,RESIDENTIAL,1-STORY 1945 & OLDER,1.0,1.0,37300.0,0.0,111700.0,NaN,NaN,856.0
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,...,RESIDENTIAL,1-STORY TWNHM - 1946 & NEWER,1.0,1.0,43500.0,0.0,130600.0,NaN,NaN,1049.0
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,...,RESIDENTIAL,1-STORY 1945 & OLDER,1.0,1.0,41100.0,0.0,123200.0,NaN,NaN,1001.0
3,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,...,RESIDENTIAL,2-STORY 1945 & OLDER,1.0,1.0,31100.0,0.0,93300.0,NaN,4850.0,1039.0
4,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,...,RESIDENTIAL,2-STORY 1946 & NEWER,1.0,1.0,64300.0,0.0,192800.0,NaN,NaN,1665.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,903205040,952,121000,30,RL,NaN,8854,Pave,NaN,Reg,...,RESIDENTIAL,1-STORY 1945 & OLDER,1.0,1.0,34600.0,0.0,103900.0,4850.0,4850.0,952.0
2576,905402060,1733,139600,20,RL,NaN,13680,Pave,NaN,IR1,...,RESIDENTIAL,1-STORY 1946 & NEWER ALL STYLES,1.0,1.0,46300.0,0.0,138800.0,NaN,NaN,1733.0
2577,909275030,2002,145000,90,RH,82.0,6270,Pave,NaN,Reg,...,RESIDENTIAL,DUPLEX - ALL STYLES AND AGES,1.0,1.0,45800.0,0.0,137400.0,NaN,NaN,2002.0
2578,907192040,1842,217500,60,RL,NaN,8826,Pave,NaN,Reg,...,RESIDENTIAL,2-STORY 1946 & NEWER,1.0,1.0,66300.0,0.0,198900.0,4850.0,4850.0,1842.0


In [5]:
tdata.columns

Index(['PID', 'GrLivArea', 'SalePrice', 'MSSubClass', 'MSZoning',
       'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond

## Columns

#### GrLivArea - keep as is
#### MSSubClass - combine into 1stry, 1plus_stry, multi then dummify
#### LotFrontage - drop
#### LotArea - keep as is
#### LotShape - binarize
#### LandContour - binarize into "level" and "other"
#### LotConfig - combine into "culdesac",  "inside", "corner"
#### Neighborhood - keep as is
#### Condition1 - cat into "Normal", "Feedr", "Artery", "Railroad", "Positive" then dummify
#### BldgType - combine 2fam conv and duplex then dummify

### MSSubClass

In [6]:
# Initial State

tdata['MSSubClass'].value_counts()

20     939
60     502
50     259
120    169
30     123
160    120
70     116
80     109
90      87
190     53
85      43
75      21
45      17
180     15
40       6
150      1
Name: MSSubClass, dtype: int64

In [7]:
# Transform Data

tdata['MSSubClass'].replace([20, 30, 40, 80, 85, 120], 'one_stry', inplace=True)
tdata['MSSubClass'].replace([45, 50, 60, 70, 75, 150, 160], 'one_plus_stry', inplace=True)
tdata['MSSubClass'].replace([90, 180, 190], 'multi_stry', inplace=True)

In [8]:
# Check Transformation

tdata['MSSubClass'].value_counts()

one_stry         1389
one_plus_stry    1036
multi_stry        155
Name: MSSubClass, dtype: int64

In [9]:
pd.get_dummies(tdata['MSSubClass'])

,multi_stry,one_plus_stry,one_stry
0,0,0,1
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0
...,...,...,...
2575,0,0,1
2576,0,0,1
2577,1,0,0
2578,0,1,0


In [10]:
# Dummify, drop 'one_stry'

MSSubClass_dums = pd.get_dummies(tdata['MSSubClass'], prefix='MSSubClass', prefix_sep='_').drop('MSSubClass_one_stry', 1)
tdata = pd.concat([tdata.drop('MSSubClass', 1), MSSubClass_dums], axis=1)

In [11]:
# Check new column

tdata.iloc[:,-2:].value_counts()

MSSubClass_multi_stry  MSSubClass_one_plus_stry
0                      0                           1389
                       1                           1036
1                      0                            155
dtype: int64

### LotFrontage (DROPPED)

### LotShape

In [12]:
# Initial State

tdata['LotShape'].value_counts()

Reg    1641
IR1     855
IR2      71
IR3      13
Name: LotShape, dtype: int64

In [13]:
# Transform Data

tdata['LotShape'].replace(['IR1', 'IR2', 'IR3'], 'IR', inplace=True)

In [14]:
# Check Transformation

tdata['LotShape'].value_counts()

Reg    1641
IR      939
Name: LotShape, dtype: int64

In [15]:
# Dummify, drop 'Reg'

LotShape_dums = pd.get_dummies(tdata['LotShape'], prefix='LotShape', prefix_sep='_').drop('LotShape_Reg', 1)
tdata = pd.concat([tdata.drop('LotShape', 1), LotShape_dums], axis=1)

In [16]:
# Check new column

tdata.iloc[:,-1:].value_counts()

LotShape_IR
0              1641
1               939
dtype: int64

### LandContour

In [17]:
# Initial State

tdata['LandContour'].value_counts()

Lvl    2337
HLS      96
Bnk      95
Low      52
Name: LandContour, dtype: int64

In [18]:
# Transform Data

tdata['LandContour'].replace(['HLS', 'Bnk', 'Low'], 'slope', inplace=True)

In [19]:
# Check Transformation

tdata['LandContour'].value_counts()

Lvl      2337
slope     243
Name: LandContour, dtype: int64

In [20]:
# Dummify, drop 'Lvl'

LandContour_dums = pd.get_dummies(tdata['LandContour'], prefix='LandContour', prefix_sep='_').drop("LandContour_Lvl", 1)
tdata = pd.concat([tdata.drop('LandContour', 1), LandContour_dums], axis=1)

In [21]:
tdata.iloc[:,-1:].value_counts()

LandContour_slope
0                    2337
1                     243
dtype: int64

### LotConfig

In [22]:
# Initial State

tdata['LotConfig'].value_counts()

Inside     1886
Corner      439
CulDSac     164
FR2          78
FR3          13
Name: LotConfig, dtype: int64

In [23]:
# Transform Data

tdata['LotConfig'].replace(['FR2', 'FR3'], 'Corner', inplace=True)

In [24]:
# Check Transformation

tdata['LotConfig'].value_counts()

Inside     1886
Corner      530
CulDSac     164
Name: LotConfig, dtype: int64

In [25]:
# Dummify, drop 'Inside'

LotConfig_dums = pd.get_dummies(tdata['LotConfig'], prefix='LotConfig', prefix_sep='_').drop('LotConfig_Inside', 1)
tdata = pd.concat([tdata.drop('LotConfig', 1), LotConfig_dums], axis=1)

In [26]:
tdata.iloc[:,-1:].value_counts()

LotConfig_CulDSac
0                    2416
1                     164
dtype: int64

### Condition1

In [27]:
# Initial State

tdata['Condition1'].value_counts()

Norm      2228
Feedr      137
Artery      81
RRAn        40
PosN        37
RRAe        26
PosA        19
RRNn         8
RRNe         4
Name: Condition1, dtype: int64

In [28]:
# Transform Data

tdata['Condition1'].replace(['RRAn', 'RRAe', 'RRNe', 'RRNn'], 'RailRd', inplace=True)
tdata['Condition1'].replace(['PosN', 'PosA'], 'Pos', inplace=True)

In [29]:
# Check Transformation

tdata['Condition1'].value_counts()

Norm      2228
Feedr      137
Artery      81
RailRd      78
Pos         56
Name: Condition1, dtype: int64

In [30]:
# Dummify, drop 'Norm'

Condition1_dums = pd.get_dummies(tdata['Condition1'], prefix='Condition1', prefix_sep='_').drop('Condition1_Norm', 1)
tdata = pd.concat([tdata.drop('Condition1', 1), Condition1_dums], axis=1)

In [31]:
tdata.iloc[:,-4:].value_counts()

Condition1_Artery  Condition1_Feedr  Condition1_Pos  Condition1_RailRd
0                  0                 0               0                    2228
                   1                 0               0                     137
1                  0                 0               0                      81
0                  0                 0               1                      78
                                     1               0                      56
dtype: int64

### BldgType

In [32]:
# Initial State

tdata['BldgType'].value_counts()

1Fam      2139
TwnhsE     202
Twnhs       98
Duplex      87
2fmCon      54
Name: BldgType, dtype: int64

In [33]:
# Transform Data

tdata['BldgType'].replace(['Duplex', '2fmCon'], '2Fam', inplace=True)

In [34]:
# Check Transformation

tdata['BldgType'].value_counts()

1Fam      2139
TwnhsE     202
2Fam       141
Twnhs       98
Name: BldgType, dtype: int64

In [35]:
# Dummify, drop '1Fam'

BldgType_dums = pd.get_dummies(tdata['BldgType'], prefix='BldgType', prefix_sep='_').drop('BldgType_1Fam', 1)
tdata = pd.concat([tdata.drop('BldgType', 1), BldgType_dums], axis=1)

In [37]:
tdata.iloc[:,-3:].value_counts()

BldgType_2Fam  BldgType_Twnhs  BldgType_TwnhsE
0              0               0                  2139
                               1                   202
1              0               0                   141
0              1               0                    98
dtype: int64